# Article figures

In [1]:
import numpy as np
import scipy.linalg as la
import cvxpy as cvx
import pandas as pd

import os
import pickle
import plotly.express as px

import hurwitz
import cvx_inference

Starting up Julia...


┌ Warning: Replacing docs for `SciMLOperators.AbstractSciMLOperator :: Union{}` in module `SciMLOperators`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `SciMLBase.sol :: Union{Tuple, Tuple{D}, Tuple{S}, Tuple{N}, Tuple{T}} where {T, N, S, D}` in module `SciMLBase`
└ @ Base.Docs docs/Docs.jl:240


## Introduction

## Methods

### [X] SCS reduction diagram

In [2]:
n = 5
A = np.array(
    [
        [-2.,0,1.,0,1.],
        [0,-2.,0,1.,0],
        [0,0,-2.,0,1.],
        [-1,0,0,-2.,0],
        [0,-1,0,0,-2.]
     ])
gamma = hurwitz.analytic_gamma(A)
E = np.abs(np.sign(A))

u, U = la.eig(gamma)

Alpha_mat = np.full(((n*(n+1))//2,n**2), 0.0)
beta_vec = np.full(((n*(n+1))//2), 0.0)
zero_inds = np.full((n**2), 0)

c1 = 0
for i in range(n):
    for j in range(i,n):

        if i!=j:
            beta_vec[c1] = 0
        else:
            beta_vec[c1] = -1/(2*u[i]).real
        c2 = 0
        for l in range(n):
            for k in range(n):
                if E[l,k] == 0: zero_inds[c2] = 1
                if (i!=j):
                    Alpha_mat[c1, c2] = cvx_inference.alphas(i, j, l, k, U, u).real
                else:
                    Alpha_mat[c1, c2] = cvx_inference.betas(i, l, k, U).real
                c2 += 1
        c1 += 1

vx = cvx.Variable(n**2)
objective = cvx.Minimize(cvx.norm(cvx.multiply(zero_inds,vx), 1))
constraints = [Alpha_mat@vx == beta_vec]
prob = cvx.Problem(objective, constraints)
probdata, chain, inverse_data = prob.get_problem_data(cvx.SCS)
bbb = probdata["b"].reshape((len(probdata["b"]), 1))
fig = px.imshow(
    probdata["A"].toarray(), 
    color_continuous_midpoint=0.0, 
    color_continuous_scale=px.colors.sequential.RdBu
    )
fig.update_traces(dict(
    
))
fig.update_xaxes(dict(
    showticklabels = False
))
fig.update_yaxes(dict(
    showticklabels = False
))
fig.show()

In [5]:
fig.write_image("figs/SCSmatrix.pdf")

### [X] Example images of reconstruction methods on 10x10

## Results

In [6]:
import experiments
n = 10
experiments.gen_rand_timeseries(Ns = [n], edge_probs=[n*0.1], mats_num=100)

100%|██████████| 100/100 [18:35<00:00, 11.16s/it]


In [2]:
import experiments
n = 10
experiments.gen_rand_timeseries(Ns = [n], edge_probs=[n*0.1], mats_num=100, saturating=True)

100%|██████████| 100/100 [11:53<00:00,  7.13s/it]


### Plots of threshold values

In [29]:
threshs = [[0, 100000, 1000],[1,100001, 500],[5, 100000, 100]]
df_list = []
for thresh in threshs:
    threshfile = "pickles/thresh_{}alpha_{}points_{}reps.pkl".format(thresh[0],thresh[1],thresh[2])
    pkl_obj = pickle.load(open(threshfile,"rb"))
    for c in range(len(pkl_obj)):
        df_list.append([thresh[0]/100, thresh[1], thresh[2], c, pkl_obj[c]])

thresh_df = pd.DataFrame(df_list, columns=["Alpha", "DataPoints", "Reps", "Conditions", "Value"])

In [30]:
plt = px.line(thresh_df, x = "Conditions", y="Value", color="Alpha")
plt.show()

### [X] Violins of accuracy in different methods

In [7]:
# import experiments
# n = 10
# experiments.run_E_inference(Ns = [n], edge_probs=[n*0.1], mats_num=100, alphas = [0.01, 0.05, 0.1])
# experiments.run_A_inference(Ns = [n], edge_probs=[n*0.1], mats_num=100, alphas = [0.01, 0.05, 0.1])

100%|██████████| 100/100 [8:59:32<00:00, 323.72s/it]  


In [10]:
N = 10
step_size = 0.1
edge_prob = N*0.1
pkl_obj = pickle.load(open("pickles/results/inferences/N{}_ss{:.2f}_ep{:.2f}_as.pkl".format(N, step_size, edge_prob), "rb"))

err_df_lst = []

for mat_num in pkl_obj.keys():
    A = pkl_obj[mat_num]["A"]
    for alpha in pkl_obj[mat_num].keys():
        if not type(alpha) is float: continue
        # for method in pkl_obj[mat_num][alpha]["A Inference"].keys():
        for method in ["Full Info + LP", "TE + LP", "No Info + LP"]:
            for rep in range(len(pkl_obj[mat_num][alpha]["A Inference"][method])):
                A_inf = pkl_obj[mat_num][alpha]["A Inference"][method][rep]
                err_df_lst.append([N, mat_num, alpha, rep, method, hurwitz.calc_mat_dist(A,A_inf)])
                
err_df = pd.DataFrame(err_df_lst, columns=["Size", "Mat", "Alpha", "Rep", "Method", "Error"])

In [11]:
plt = px.violin(err_df[(err_df["Alpha"] == 0.01) & (err_df["Method"].isin(["Full Info + LP", "TE + LP", "No Info + LP"]))], x = "Method", y = "Error", color="Method", box=True, range_y=[0,0.25], points="outliers")
plt.show()

In [ ]:
plt.write_image("figs/SCSmatrix.svg")

### [ ] Violins of accuracy when saturating

In [14]:
import importlib
importlib.reload(experiments)

<module 'experiments' from '/home/ianxul/Documents/neurolab/CorTe-Inference/experiments.py'>

In [3]:
import experiments
n = 10
# experiments.gen_rand_timeseries(Ns = [n], edge_probs=[n*0.1], mats_num=100, saturating=True)
# experiments.run_E_inference(Ns = [n], edge_probs=[n*0.1], mats_num=100, alphas = [0.01], saturating=True)
experiments.run_A_inference(Ns = [n], edge_probs=[n*0.1], mats_num=100, alphas = [0.01], saturating = True)

  0%|          | 0/100 [00:00<?, ?it/s]/home/ianxul/Documents/neurolab/CorTe-Inference/corr_inference.py:87: ComplexWarning: Casting complex values to real discards the imaginary part
  Alpha_mat[k,r] = alphas(i,j,ll,kk, U, u)
/home/ianxul/Documents/neurolab/CorTe-Inference/corr_inference.py:92: ComplexWarning: Casting complex values to real discards the imaginary part
  beta_vec[k] = -betas(i,j, U, u)
/home/ianxul/Documents/neurolab/CorTe-Inference/corr_inference.py:15: ComplexWarning: Casting complex values to real discards the imaginary part
  A_sol[i,j] = betas(i,j, U, u) + sum(x_sol*np.array([alphas(i,j,l,k, U, u) for l in range(n) for k in range(l+1,n)]))
100%|██████████| 100/100 [00:18<00:00,  5.36it/s]


In [14]:
N = 10
step_size = 0.1
edge_prob = N*0.1
pkl_obj = pickle.load(open("pickles/results/inferences/N{}_ss{:.2f}_ep{:.2f}_as_sat.pkl".format(N, step_size, edge_prob), "rb"))

err_df_lst = []

for mat_num in pkl_obj.keys():
    A = pkl_obj[mat_num]["A"]
    for alpha in pkl_obj[mat_num].keys():
        if not type(alpha) is float: continue
        # for method in pkl_obj[mat_num][alpha]["A Inference"].keys():
        for method in ["Full Info + LP", "TE + LP", "No Info + LP"]:
            for rep in range(len(pkl_obj[mat_num][alpha]["A Inference"][method])):
                A_inf = pkl_obj[mat_num][alpha]["A Inference"][method][rep]
                err_df_lst.append([N, mat_num, alpha, rep, method, np.sum(A*A_inf)/(np.sqrt(np.sum(A*A))*(np.sqrt(np.sum(A_inf*A_inf))))])
                
err_df = pd.DataFrame(err_df_lst, columns=["Size", "Mat", "Alpha", "Rep", "Method", "Error"])

In [15]:
plt = px.violin(err_df[(err_df["Alpha"] == 0.01) & (err_df["Method"].isin(["Full Info + LP", "TE + LP", "No Info + LP"]))], x = "Method", y = "Error", color="Method", box=True, points="outliers")
plt.show()

In [18]:
XX = 0

In [37]:
mat_num = XX
XX += 1
alpha = 0.01
hurwitz.make_triple_imshow(pkl_obj[mat_num]["A"], 3*pkl_obj[mat_num][alpha]["A Inference"]["Full Info + LP"][0], pkl_obj[mat_num][alpha]["E Inference"][0], ["A", "TE", "E"])

In [ ]:
## Show also a triple imshow with the correlation matrices produced by the data and the methods just to make sure they're staying within the solution set

### [ ] Plot of accuracy/alignment vs. edge probability

### [ ] An example with external noise ? 

### [ ] Examples with non-linear (saturating) systems
Imagino que para este caso podría ser una figura que muestre ejemplos particulares de la matriz de conectividad, la reconstrucción, y los valores de alineación 

In [2]:
from idtxl.data import Data
from idtxl.multivariate_te import MultivariateTE
from idtxl.visualise_graph import plot_network

In [3]:
A = np.array([
    [-3,+0,+1,+0,+0],
    [+1,-3,+0,+0,+0],
    [+1,+0,-3,+0,+0],
    [+1,+0,+0,-3,+0],
    [+1,+0,+0,+1,-3]
])

In [4]:
data_np = hurwitz.run_process_jl(A, 10000, saturating = True)
data_np

array([[ 0.        ,  0.34175807,  0.43641685, ...,  1.05352908,
         0.83783908,  0.59225695],
       [ 0.        ,  0.13972978,  0.11134983, ...,  0.0674741 ,
         0.22232671, -0.03122421],
       [ 0.        ,  0.17138216, -0.05221145, ..., -0.06509255,
        -0.03716864,  0.15604022],
       [ 0.        , -0.03045802, -0.11526899, ...,  2.50428307,
         2.1416177 ,  2.15079094],
       [ 0.        , -0.03097437,  0.26257358, ...,  1.47131707,
         1.52052748,  1.34767256]])

In [19]:
import importlib
importlib.reload(te_inference)

<module 'te_inference' from '/home/ianxul/Documents/neurolab/CorTe-Inference/te_inference.py'>

In [47]:
import te_inference
E_est, E_vals = te_inference.perform_inference_max(data_np[:, :100000], alpha = 0.005, test_reps = 1000, gpu = True, limit = 25)

In [ ]:
E_est, E_vals

(array([[1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 0, 1]]),
 array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [-0.00645853,  0.        ,  0.        ,  0.        ,  0.        ],
        [-0.0080647 ,  0.        ,  0.        ,  0.        ,  0.        ]]))

In [42]:
import te_estimate
te_vals = (te_estimate.generate_TE_mat(data_np, gpu = True).T)
np.around(te_vals, 4)

array([[1.    , 0.0767, 0.0592, 0.0706, 0.098 ],
       [0.0988, 1.    , 0.0787, 0.075 , 0.0944],
       [0.0648, 0.0699, 1.    , 0.0673, 0.097 ],
       [0.0957, 0.0719, 0.0756, 1.    , 0.095 ],
       [0.1331, 0.1047, 0.1157, 0.1212, 1.    ]])

In [43]:
te_vals_cond = (te_estimate.generate_TE_mat(data_np, E_est, gpu = True).T)
np.around(te_vals_cond, 4)

array([[1.    , 0.1108, 0.1081, 0.1075, 0.098 ],
       [0.1237, 1.    , 0.0822, 0.0846, 0.1237],
       [0.1108, 0.1135, 1.    , 0.108 , 0.097 ],
       [0.1261, 0.0881, 0.0828, 1.    , 0.1231],
       [0.0672, 0.0706, 0.0671, 0.0739, 1.    ]])

In [49]:
A_est = cvx_inference.find_sol_cvx(E_est, data_np)

In [50]:
px.imshow(A_est)

In [5]:
dt = Data(data_np, "ps")
network_analysis = MultivariateTE()
settings = {'cmi_estimator': 'OpenCLKraskovCMI',
            'max_lag_sources': 1,
            'min_lag_sources': 1,
            'n_perm_max_stat': 100, # Maximum test
            'alpha_max_stat': 0.05,
            'n_perm_min_stat': 100, # Minimum test
            'alpha_min_stat': 0.05,
            'n_perm_omnibus': 100, # Omnibus
            'alpha_omnibus': 0.5,
            'n_perm_max_seq': 100,
            'alpha_max_seq': 0.1,
            'permute_in_time': True}
results = network_analysis.analyse_network(settings=settings, data=dt)

Adding data with properties: 5 processes, 100001 samples, 1 replications
overwriting existing data

####### analysing target with index 0 from list [0, 1, 2, 3, 4]

Target: 0 - testing sources [1, 2, 3, 4]

---------------------------- (1) include target candidates
candidate set: [(0, 1)]
testing candidate: (0, 1) maximum statistic, n_perm: 100

---------------------------- (2) include source candidates
candidate set: [(1, 1), (2, 1), (3, 1), (4, 1)]
testing candidate: (2, 1) maximum statistic, n_perm: 100
